# Exploring Hacker News Posts

Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") receive votes and comments, similar to reddit.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that we have reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that didn't receive any comments and then randomly sampling from the remaining submissions. Below are descriptions of the columns:
* **id**: the unique identifier from Hacker News for the post.
* **title**: the title of the post.
* **url**: the URL that the posts links to, if the post has a URL.
* **num_points**: the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes.
* **num_comments**: the number of comments on the post.
* **author**: the username of the person who submitted the post.
* **created_at**: the date and time of the post's submission.

We're specifically interested in posts with titles that begin with either **Ask HN** or **Show HN**. Users submit **Ask HN** posts to ask the Hacker News community a specific question.
Likewise, users submit **Show HN** posts to show the Hacker News community a project, product, or just something interesting.

We'll compare these two types of posts to determine the following:

* Do **Ask HN** or **Show HN** receive more comments on average?
* Do posts created at a certain time receive more comments on average?

## Introduction

Let's start by importing the libraries we need and reading the dataset into a list of lists.



In [1]:
from csv import reader

opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

Let's display some rows to check that everything is working correctly.

In [2]:
display(hn[:6])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
 

## Removing Headers from a List of Lists

As you may have noticed, when we displayed the first five rows of `hn`, the first list in the inner lists contains the column headers, and the lists after contain the data for one row. In order to analyze our data, we need to first remove the row containing the column headers.

Let's remove that first row next.

In [3]:
headers = hn[0]
hn = hn[1:]

display(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

Let's display the first five rows again to verify that we removed the header row properly.

In [4]:
display(hn[:5])

[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

## Extracting Ask HN and Show HN Posts

Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [5]:
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1].lower()
    
    if title.startswith("ask hn"):
        ask_posts.append(post)
    elif title.startswith("show hn"):
        show_posts.append(post)
    else:
        other_posts.append(post)

Let's check the number of posts in `ask_posts`, `show_posts` and `other_posts`.

In [6]:
print(f"Ask posts: {len(ask_posts)}\nShow posts: {len(show_posts)}\nOther posts: {len(other_posts)}")

Ask posts: 1744
Show posts: 1162
Other posts: 17194


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

We seperated the **ask posts** and the **show posts** into two lists of lists named `ask_posts` and `show_posts`.

Below are the first five rows in the `ask_posts` list of lists:

In [7]:
display(ask_posts[:6])

[['12296411',
  'Ask HN: How to improve my personal website?',
  '',
  '2',
  '6',
  'ahmedbaracat',
  '8/16/2016 9:55'],
 ['10610020',
  'Ask HN: Am I the only one outraged by Twitter shutting down share counts?',
  '',
  '28',
  '29',
  'tkfx',
  '11/22/2015 13:43'],
 ['11610310',
  'Ask HN: Aby recent changes to CSS that broke mobile?',
  '',
  '1',
  '1',
  'polskibus',
  '5/2/2016 10:14'],
 ['12210105',
  'Ask HN: Looking for Employee #3 How do I do it?',
  '',
  '1',
  '3',
  'sph130',
  '8/2/2016 14:20'],
 ['10394168',
  'Ask HN: Someone offered to buy my browser extension from me. What now?',
  '',
  '28',
  '17',
  'roykolak',
  '10/15/2015 16:38'],
 ['10284812',
  'Ask HN: Limiting CPU, memory, and I/O usage on a program for testing',
  '',
  '2',
  '1',
  'zatkin',
  '9/26/2015 23:23']]

Below are the first five rows in the `show_posts` list of lists.

In [8]:
display(show_posts[:6])

[['10627194',
  'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform',
  'https://iot.seeed.cc',
  '26',
  '22',
  'kfihihc',
  '11/25/2015 14:03'],
 ['10646440',
  'Show HN: Something pointless I made',
  'http://dn.ht/picklecat/',
  '747',
  '102',
  'dhotson',
  '11/29/2015 22:46'],
 ['11590768',
  'Show HN: Shanhu.io, a programming playground powered by e8vm',
  'https://shanhu.io',
  '1',
  '1',
  'h8liu',
  '4/28/2016 18:05'],
 ['12178806',
  'Show HN: Webscope  Easy way for web developers to communicate with Clients',
  'http://webscopeapp.com',
  '3',
  '3',
  'fastbrick',
  '7/28/2016 7:11'],
 ['10872799',
  'Show HN: GeoScreenshot  Easily test Geo-IP based web pages',
  'https://www.geoscreenshot.com/',
  '1',
  '9',
  'kpsychwave',
  '1/9/2016 20:45'],
 ['11237259',
  'Show HN: Run with Mark (Runkeeper only)',
  'http://runwithmark.github.io/#/',
  '3',
  '3',
  'ecesena',
  '3/7/2016 5:17']]

Next, let's determine if ask posts or show posts receive more comments on average.

In [9]:
total_ask_comments = 0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments / len(ask_posts)

Below is the average number of comments on ask posts:

In [10]:
print(round(avg_ask_comments, 2))

14.04


In [11]:
total_show_comments = 0

for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments

avg_show_comments = total_show_comments /  len(show_posts)

Below is the average number of comments on show posts:

In [12]:
print(round(avg_show_comments, 2))

10.32


As we can see, ask posts receive more comments on average. We can deduce that this is so because a question require an answer, a solution that is provided by the community and often a problem or a simple question can have several answers.

## Finding the Number of Ask Posts and Comments by Hour Created

We determined that, on average, ask posts receive more comments than show posts. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain *time* are more likely to attract comments. We'll use the following steps to perform this analysis:
1. Calculate the number of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

Now we'll work on the first step - calculating the number of ask posts and comments by hour created.

In [13]:
import datetime as dt

result_list = []

for post in ask_posts:
    created_at = post[6]
    num_comments = int(post[4])
    
    result_list.append([created_at, num_comments])

In [14]:
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    post_dt = row[0]
    num_comments = row[1]
    
    parsed_dt= dt.datetime.strptime(post_dt, "%m/%d/%Y %H:%M")
    post_hr = parsed_dt.strftime("%H")
    
    if post_hr in counts_by_hour:
        counts_by_hour[post_hr] += 1
        comments_by_hour[post_hr] += num_comments
    else:
        counts_by_hour[post_hr] = 1
        comments_by_hour[post_hr] = num_comments

Below are the number of ask posts by hour created.

In [15]:
display(counts_by_hour)

{'09': 45,
 '13': 85,
 '10': 59,
 '14': 107,
 '16': 108,
 '23': 68,
 '12': 73,
 '17': 100,
 '15': 116,
 '21': 109,
 '20': 80,
 '02': 58,
 '18': 109,
 '03': 54,
 '05': 46,
 '19': 110,
 '01': 60,
 '22': 71,
 '08': 48,
 '04': 47,
 '00': 55,
 '06': 44,
 '07': 34,
 '11': 58}

Below are the number of comments by hour created.

In [16]:
display(comments_by_hour)

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

## Calculating the Average Number of Comments for Ask HN Posts by Hour

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

Let's create a list of lists in which the first element is the hour and the second element is the average number of comments per post.

In [17]:
avg_by_hour = []

for k, v in comments_by_hour.items():
    avg_comments = round(v / counts_by_hour[k], 2)
    
    avg_by_hour.append([k, avg_comments])

Let's display the average number of comments for posts created during each hour of the day.

In [18]:
display(avg_by_hour)

[['09', 5.58],
 ['13', 14.74],
 ['10', 13.44],
 ['14', 13.23],
 ['16', 16.8],
 ['23', 7.99],
 ['12', 9.41],
 ['17', 11.46],
 ['15', 38.59],
 ['21', 16.01],
 ['20', 21.52],
 ['02', 23.81],
 ['18', 13.2],
 ['03', 7.8],
 ['05', 10.09],
 ['19', 10.8],
 ['01', 11.38],
 ['22', 6.75],
 ['08', 10.25],
 ['04', 7.17],
 ['00', 8.13],
 ['06', 9.02],
 ['07', 7.85],
 ['11', 11.05]]

## Sorting and Printing Values from a List of Lists

Although we now have the results we need, this format makes it difficult to indetify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

First, we need to create a list that equals `avg_by_hour` with swapped columns.

In [19]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

Below is the list with swapped columns

In [20]:
display(swap_avg_by_hour)

[[5.58, '09'],
 [14.74, '13'],
 [13.44, '10'],
 [13.23, '14'],
 [16.8, '16'],
 [7.99, '23'],
 [9.41, '12'],
 [11.46, '17'],
 [38.59, '15'],
 [16.01, '21'],
 [21.52, '20'],
 [23.81, '02'],
 [13.2, '18'],
 [7.8, '03'],
 [10.09, '05'],
 [10.8, '19'],
 [11.38, '01'],
 [6.75, '22'],
 [10.25, '08'],
 [7.17, '04'],
 [8.13, '00'],
 [9.02, '06'],
 [7.85, '07'],
 [11.05, '11']]

Finally, we need to sort the list of lists

In [21]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

### Top 5 Hours for Ask Posts Comments

In [24]:
for row in sorted_swap[:6]:
    hr = row[1]
    hr_dt = dt.datetime.strptime(hr, "%H")
    hr_dt = hr_dt.strftime("%H:%M")
    
    print(f"{hr_dt}: {row[0]} average comments per post")

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.8 average comments per post
21:00: 16.01 average comments per post
13:00: 14.74 average comments per post


## Conclusion

After having analyzed the data from Hacker News we can conclude two things:
1. The first thing is that the **Ask HN** posts receive more comments than **Show HN** posts.
2. Second, we conclude that we have a higher chance of receiving comments if we publish the post at a certain time, like the hours mentioned above.